In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
import re

In [111]:
# Download stopwords for NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dzaytsev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
df = pd.read_csv('yelp.csv')

In [66]:
df

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0


In [68]:
df = df[['text', 'stars']]

In [70]:
df['label'] = df['stars'].apply(lambda x: 'Sad' if x < 3 else 'Happy')

/var/folders/4w/nfkvhvrs0sxd55472ytdz0nh0000gp/T/ipykernel_8295/3819513822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['stars'].apply(lambda x: 'Sad' if x < 3 else 'Happy')


In [76]:
df.drop(['stars'], axis=1, inplace=True)

/var/folders/4w/nfkvhvrs0sxd55472ytdz0nh0000gp/T/ipykernel_8295/3206604362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['stars'], axis=1, inplace=True)


In [78]:
df

,text,label
0,My wife took me here on my birthday for breakf...,Happy
1,I have no idea why some people give bad review...,Happy
2,love the gyro plate. Rice is so good and I als...,Happy
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",Happy
4,General Manager Scott Petello is a good egg!!!...,Happy
...,...,...
9995,First visit...Had lunch here today - used my G...,Happy
9996,Should be called house of deliciousness!\n\nI ...,Happy
9997,I recently visited Olive and Ivy for business ...,Happy
9998,My nephew just moved to Scottsdale recently so...,Sad


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10000 non-null  object
 1   label   10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [93]:
df['label'].value_counts()

label
Happy    8324
Sad      1676
Name: count, dtype: int64

In [103]:
#Preprocess the text data
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\d', '', text)  # Remove numbers
    stop_words = set(stopwords.words('english'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])
    return text

In [105]:
a = preprocess_text(df['text'][0])

In [107]:
a

'wife took birthday breakfast excellent weather perfect made sitting outside overlooking grounds absolute pleasure waitress excellent food arrived quickly semi busy saturday morning looked like place fills pretty quickly earlier get better favor get bloody mary phenomenal simply best ever pretty sure use ingredients garden blend fresh order amazing everything menu looks excellent white truffle scrambled eggs vegetable skillet tasty delicious came pieces griddled bread amazing absolutely made meal complete best toast ever anyway wait go back'

In [109]:
df['text'][0]

'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'

In [113]:
message = df['text'].apply(preprocess_text)
label = df['label'].map({'Happy': 1, 'Sad': 0})

In [121]:
#Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(message, label, test_size=0.2, random_state=42)

In [123]:
#Vectorize the text using TF-IDF (Term Frequency-Inverse Document Frequency)
vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [125]:
X_train_tfidf

<8000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 373091 stored elements in Compressed Sparse Row format>

In [127]:
X_test_tfidf

<2000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 88954 stored elements in Compressed Sparse Row format>

In [129]:
#Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [131]:
print(f"Testing score is {model.score(X_test_tfidf, y_test)} and TrainingScore is {model.score(X_train_tfidf, y_train)} ")

Testing score is 0.887 and TrainingScore is 0.911125 


In [143]:
def get_intent(pred):
    if prediction[0] == 1:
        return 'HAPPY :)'
    else:
        return 'SAD :('

In [150]:
user_input = input("Enter a message to classify: ")
processed_input = preprocess_text(user_input)  # Preprocess the user input
input_tfidf = vectorizer.transform([processed_input])  # Transform the input to the same TF-IDF format
prediction = model.predict(input_tfidf)

print("This message intent is:", get_intent(prediction))


Enter a message to classify:  I feel good today


This message intent is: HAPPY :)


In [152]:
user_input = input("Enter a message to classify: ")
processed_input = preprocess_text(user_input)  # Preprocess the user input
input_tfidf = vectorizer.transform([processed_input])  # Transform the input to the same TF-IDF format
prediction = model.predict(input_tfidf)

print("This message intent is:", get_intent(prediction))

Enter a message to classify:  This is the worst day in my life!


This message intent is: SAD :(
